In [1]:
using Pkg
Pkg.activate("C:/Users/lisah/Documents/Repos/ma-code")

  Activating project at `C:\Users\lisah\Documents\Repos\ma-code`


In [2]:
include("c:/Users/lisah/Documents/Repos/ma-code/src/src.jl")
using .Src

In [3]:
using JLD2, Distributions

## Computation of Sobol Indices

In [ ]:
N = 1000
t_fixed = true
t_end = 50.0
M = Int(t_end * 10) # pick M solution trajectory time points for computing indices

500

### analysis for different parameter densities

In [5]:
# define densities that are consistent
dens_m = Uniform(0.45-0.45*0.25,0.45+0.45*0.25)
dens_n0 = Uniform(0.0,2.0)
dens_w0 = Uniform(0.0,2.0);

#### a left of bif pt

In [6]:
dens_a = Uniform(0.0,0.84)
sobol_n, sobol_w = compute_sobol_indices(N, dens_a, dens_m, dens_w0, dens_n0, M, t_fixed, t_end)

Calculating indices for 4 parameters ... 100%|███████████| Time: 0:00:00
Calculating indices for 4 parameters ... 100%|███████████| Time: 0:00:00
Calculating indices for 4 parameters ... 100%|███████████| Time: 0:00:00
Calculating indices for 4 parameters ... 100%|███████████| Time: 0:00:00
Calculating indices for 4 parameters ... 100%|███████████| Time: 0:00:00
Calculating indices for 4 parameters ... 100%|███████████| Time: 0:00:00
Calculating indices for 4 parameters ... 100%|███████████| Time: 0:00:00
Calculating indices for 4 parameters ... 100%|███████████| Time: 0:00:00
Calculating indices for 4 parameters ... 100%|███████████| Time: 0:00:00
Calculating indices for 4 parameters ... 100%|███████████| Time: 0:00:00
Calculating indices for 4 parameters ... 100%|███████████| Time: 0:00:00
Calculating indices for 4 parameters ... 100%|███████████| Time: 0:00:00
Calculating indices for 4 parameters ... 100%|███████████| Time: 0:00:00
Calculating indices for 4 parameters ... 100%|█████

(Dict{Symbol, Array}[Dict(:firstorder => [0.0, 0.0, 0.0, 0.3309733611776091], :secondorder_conf => Union{Missing, Float64}[missing 0.0 0.0 0.0; missing missing 0.0 0.0; missing missing missing 0.0; missing missing missing missing], :totalorder_conf => [0.0, 0.0, 0.0, 1.6363166450338777], :totalorder => [0.0, 0.0, 0.0, 0.3309733611776091], :firstorder_conf => [0.0, 0.0, 0.0, 6.306642579650103], :secondorder => Union{Missing, Float64}[missing 0.0 0.0 0.0; missing missing 0.0 0.0; missing missing missing 0.0; missing missing missing missing]), Dict(:firstorder => [0.0005243729367465826, -0.030338450108348, -0.17683193255753485, 0.38789723297031664], :secondorder_conf => Union{Missing, Float64}[missing 0.032260974669381215 0.02712767673115689 0.05389113289593911; missing missing 1.36673318116816 2.660880729145155; missing missing missing 19.770811384138902; missing missing missing missing], :totalorder_conf => [7.342611827411561e-6, 0.022424827617176252, 0.747130830397808, 8.10751873543891

In [7]:
# store
@save "C:/Users/lisah/Documents/Repos/ma-code/temp_saves/global_inits/dl/sobol_n_$(N)_$(t_end)_$M.jld2" sobol_n
@save "C:/Users/lisah/Documents/Repos/ma-code/temp_saves/global_inits/dl/sobol_w_$(N)_$(t_end)_$M.jld2" sobol_w

#### a around bif pt

In [17]:
dens_a = Uniform(0.84,0.94)
sobol_n, sobol_w = compute_sobol_indices(N, dens_a, dens_m, dens_w0, dens_n0, M, t_fixed, t_end)

UndefVarError: UndefVarError: `compute_sobol_indices` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [18]:
# store
@save "C:/Users/lisah/Documents/Repos/ma-code/temp_saves/global_inits/dm/sobol_n_$(N)_$(t_end)_$M.jld2" sobol_n
@save "C:/Users/lisah/Documents/Repos/ma-code/temp_saves/global_inits/dm/sobol_w_$(N)_$(t_end)_$M.jld2" sobol_w

UndefVarError: UndefVarError: `sobol_n` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

#### a right of bif pt

In [19]:
dens_a = Uniform(0.94, 2.0)
sobol_n, sobol_w = compute_sobol_indices(N, dens_a, dens_m, dens_w0, dens_n0, M, t_fixed, t_end)

UndefVarError: UndefVarError: `compute_sobol_indices` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [20]:
# store
@save "C:/Users/lisah/Documents/Repos/ma-code/temp_saves/global_inits/dr/sobol_n_$(N)_$(t_end)_$M.jld2" sobol_n
@save "C:/Users/lisah/Documents/Repos/ma-code/temp_saves/global_inits/dr/sobol_w_$(N)_$(t_end)_$M.jld2" sobol_w

UndefVarError: UndefVarError: `sobol_n` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

#### müll

n
- as expected, total index always greater than first order index
- initial conditions rather insensitive
- a is most sensitive parameter
- si_a + si_m = 1 and total indices same as first order indices -> no significant interactions

w
- TOI significantly higher than FOI -> apparently strong parameter interactions
- qualitative behavior of FOI similar to n case
- w0 least sensitive
- n0 somehow sensitive
- a most sensitive, m also quite sensitive

Q: in local sensitivity  analysis w did not exhibit any sensitivity